In [1]:
import pandas as pd

In [2]:
data = pd.read_excel('run-data.xlsx')

In [3]:
data.shape

(12, 33)

In [4]:
# data.drop(len(data)-1, inplace=True)

In [5]:
# data.to_excel('../run-data.xlsx', index=False)

In [6]:
data.head()

,rollout,waitingTime,time,rew,Ep_num_train_rollouts,Ep_rollout_length,Ep_eval_freq,Ep_eval_num_eps,Ep_max_ep_steps,Ep_generation_ep_steps,...,P_value_loss_coef,P_hidden_layer_size,R_rule_set,R_rule_set_params,En_shape,En_rush_hour,En_uniform_generation_probability,M_reward_interpolation,M_state_interpolation,P_num_workers
0,250,513.509167,38.621479,1139.533778,300,2048,50,25,1250,1000,...,1,32,cycle,"{'cycle_length': 20, 'NS_mult': 1.0, 'EW_mult'...","[3, 3]",False,0.05,0,0,8
1,300,511.363333,40.813751,1141.940111,300,2048,50,25,1250,1000,...,1,32,cycle,"{'cycle_length': 20, 'NS_mult': 1.0, 'EW_mult'...","[3, 3]",False,0.05,0,0,8
2,308,491.545833,38.034952,1145.147556,300,2048,50,25,1250,1000,...,1,32,cycle,"{'cycle_length': 20, 'NS_mult': 1.0, 'EW_mult'...","[3, 3]",False,0.05,0,0,8
3,250,520.530833,38.971926,1138.732444,300,2048,50,25,1250,1000,...,1,32,cycle,"{'cycle_length': 20, 'NS_mult': 1.0, 'EW_mult'...","[3, 3]",False,0.05,0,0,8
4,250,488.763333,39.428385,1146.711444,300,2048,50,25,1250,1000,...,1,32,cycle,"{'cycle_length': 20, 'NS_mult': 1.0, 'EW_mult'...","[3, 3]",False,0.05,0,0,8


# Notes
- rollout_length: at least 1024 (2048 is better)
- gae_tau: at most 0.9 (maybe lower)
- entropy_weight: at least 0.1 (test higher)
- minibatch_size: at least 128 (maybe higher)
- optimization_epochs: 5 but 10 is very close
- ppo_ratio_clip: 0.3 but 0.2 is close
- clip_grads: true
- value_loss_coef: 1.0
- reward_interpolation: 0 
- state_interpolation: anything
- warmup_ep_steps: anything (0)


- NS_mult: 1.0 or 1.5
- phase_end_offset: any (0 - 100)
- multi intersection cycle length: 20
- (best combo 1.5, 100, 20)

In [8]:
data[data['Ep_warmup_ep_steps'] == 0]['waitingTime'].mean()

505.47277777777776

In [9]:
data[data['Ep_warmup_ep_steps'] == 50]['waitingTime'].mean()

509.30361111111114

In [10]:
data[data['Ep_warmup_ep_steps'] == 100]['waitingTime'].mean()

510.13444444444445

In [11]:
data[data['Ep_warmup_ep_steps'] == 200]['waitingTime'].mean()

520.2569444444445

In [11]:
data[data['M_state_interpolation'] == 1]['waitingTime'].mean()

533.9041666666666